In [ ]:
from langchain.prompts import PromptTemplate

planner_prompt = PromptTemplate.from_template(
    "You are the Planner agent. Break down this task into steps:\n{input}"
)

research_prompt = PromptTemplate.from_template(
    "You are the Research agent. Use your tools to gather data for this plan:\n{input}"
)

coder_prompt = PromptTemplate.from_template(
    "You are the Coder agent. Write Python matplotlib code to generate the requested chart.\nUse this research:\n{input}"
)

executor_prompt = PromptTemplate.from_template(
    "You are the Executor agent. Run the Python code provided and return printed output or errors:\n{input}"
)


In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.tools.tavily_search import TavilySearchResults
from langchain.agents import create_react_agent, AgentExecutor
from langchain_experimental.utilities import PythonREPL
from langgraph.graph import StateGraph
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langgraph.types import Command

token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")

# Load environment variables
load_dotenv()
TAVILY_API_KEY = os.getenv("TAVILY_APIKEY")


# 1. Define shared AgentState
class AgentState(BaseModel):
    input: str
    plan: Optional[str] = None
    research: Optional[str] = None
    search_results: Optional[str] = None
    output: Optional[str] = None
    execution_result: Optional[str] = None


# 2. Initialize LLM
llm = AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    model= "gpt-4o-mini"
)


# 3. Define Tavily web search tool using @tool decorator
@tool("tavily_web_search")
def web_search(query: str) -> str:
    """
    Perform a web search using Tavily with the given query string.
    Returns combined search results as a text string.
    """
    print("🕸️ Tavily Web Search tool called with query:", query)
    try:
        results = TavilySearchResults(max_results=4, tavily_api_key=TAVILY_API_KEY).invoke(query)
        if results:
            # Combine first few results content or just first one
            combined = "\n\n".join(r["content"] for r in results)
            return combined
        else:
            return "No results found."
    except Exception as e:
        print("Error in Tavily search:", e)
        return f"Error in Tavily search: {e}"


# 4. Wrap PythonREPL as tool using @tool
python_repl_instance = PythonREPL()

@tool("python_repl")
def python_repl_tool(code: str) -> str:
    """
    Execute Python code using a Python REPL environment.
    Returns the printed output or error messages from code execution.
    """
    print("🐍 PythonREPL tool running code...")
    return python_repl_instance.run(code)


# 5. Create agents and executors
planner_agent = create_react_agent(llm, tools=[], prompt=planner_prompt)
research_agent = create_react_agent(llm, tools=[web_search], prompt=research_prompt)
coder_agent = create_react_agent(llm, tools=[], prompt=coder_prompt)
executor_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=executor_prompt)


planner_executor = AgentExecutor(agent=planner_agent, tools=[], verbose=True)
research_executor = AgentExecutor(agent=research_agent, tools=[web_search], verbose=True)
coder_executor = AgentExecutor(agent=coder_agent, tools=[], verbose=True)
executor_executor = AgentExecutor(agent=executor_agent, tools=[python_repl_tool], verbose=True)


# 6. Define node functions with handoffs

def planner_node(state: AgentState):
    """
    Planner agent: Break down the user's input task into clear actionable steps.
    Updates state.plan with the generated plan.
    Returns a command to handoff control to the Research agent.
    """
    print("🧠 Planner Agent Running...")
    result = planner_executor.invoke({"input": f"Break this task into clear steps: {state.input}"})
    new_state = state.copy(update={"plan": result["output"]})
    return Command.goto("researcher", state=new_state)

def research_node(state: AgentState):
    """
    Research agent: Use the plan or input to query Tavily web search tool.
    Updates state.research and state.search_results with search data.
    Returns a command to handoff control to the Coder agent.
    """    
    print("🔍 Research Agent Running...")
    # Use plan or input as query
    query = state.plan or state.input
    result = research_executor.invoke({"input": query})
    new_state = state.copy(update={"research": result["output"], "search_results": result["output"]})
    return Command.goto("coder", state=new_state)

def coder_node(state: AgentState):
    """
    Coder agent: Generate Python matplotlib code to draw the requested chart
    using the research data from state.research.
    Updates state.output with the generated Python code.
    Returns a command to handoff control to the Executor agent.
    """
    print("💻 Coder Agent Running...")
    prompt = (
        "Write complete Python code using matplotlib to draw the requested chart.\n"
        "Use this research:\n"
        f"{state.research}\n"
        "Include data inline, and end with plt.show() to display the chart.\n"
        "Only output the code without explanations."
    )
    result = coder_executor.invoke({"input": prompt})
    new_state = state.copy(update={"output": result["output"]})
    return Command.goto("executor", state=new_state)

def executor_node(state: AgentState):
    """
    Executor agent: Run the Python code stored in state.output using PythonREPL.
    Updates state.execution_result with the output or error messages.
    Returns the final updated state ending the multi-agent flow.
    """
    print("⚙️ Executor Agent Running...")
    try:
        result = executor_executor.invoke({"input": state.output})
        exec_output = result.get("output", "")
    except Exception as e:
        exec_output = f"Error during execution: {e}"
    new_state = state.copy(update={"execution_result": exec_output})
    # End of chain, no next node
    return new_state


# 7. Build and compile graph

builder = StateGraph(AgentState)
builder.add_node("planner", planner_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", coder_node)
builder.add_node("executor", executor_node)

builder.set_entry_point("planner")
builder.set_finish_point("executor")

graph = builder.compile()


# 8. Run the multi-agent graph

if __name__ == "__main__":
    task = "Draw a line chart of India's GDP from 2013 to 2023"

    initial_state = AgentState(input=task)

    final_state = graph.invoke(initial_state)

    print("\n=== Generated Python Code ===\n")
    print(final_state.output)

    print("\n=== Execution Result (prints/errors) ===\n")
    print(final_state.execution_result)


In [11]:
import os
from typing import Literal
from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain_experimental.utilities import PythonREPL
from langchain.tools import Tool

from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState, END, Graph
from langgraph.types import Command

# Load environment variables
load_dotenv()

token_provider = get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")


# --- Initialize AzureChatOpenAI LLM ---
llm = AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider=token_provider,
    model="gpt-4o-mini"
)

# --- Tavily Tool ---
TAVILY_API_KEY = os.getenv("TAVILY_APIKEY")
assert TAVILY_API_KEY, "Set TAVILY_APIKEY in your .env"

from langchain.tools import BaseTool

import requests

class TavilySearchTool(BaseTool):
    name = "tavily_search"
    description = "Search the web using Tavily for factual data."

    def _run(self, query: str) -> str:
        # Real Tavily API call
        headers = {"Authorization": f"Bearer {TAVILY_API_KEY}"}
        payload = {"query": query, "max_results": 4}
        response = requests.post("https://api.tavily.com/v1/search", json=payload, headers=headers)
        response.raise_for_status()
        data = response.json()

        # Extract content from results and format as string
        results = data.get("results", [])
        output = "\n".join([f"- {item.get('content','')}" for item in results])
        return output

    async def _arun(self, query: str) -> str:
        raise NotImplementedError("Async not implemented")

tavily_tool = TavilySearchTool()

# --- Python REPL Tool ---
python_repl = PythonREPL()

def python_repl_func(code: str) -> str:
    try:
        return python_repl.run(code)
    except Exception as e:
        return f"Error executing code: {e}"

python_repl_tool = Tool(
    name="python_repl",
    func=python_repl_func,
    description="Execute Python code and return the printed output or errors."
)

# --- Helper function to detect final answer ---
def get_next_node(last_message: BaseMessage, goto: str):
    if "FINAL ANSWER" in last_message.content:
        return END
    return goto

def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

# --- Agent Prompts ---
planner_prompt = make_system_prompt(
    "You are the Planner agent. Break down the user task into clear steps. "
    "Work with Research, Coder, and Executor colleagues."
)

research_prompt = make_system_prompt(
    "You can only do research. You are working with Planner, Coder, and Executor colleagues."
)

coder_prompt = make_system_prompt(
    "You can only write Python matplotlib code to generate charts. "
    "You are working with Planner, Research, and Executor colleagues."
)

executor_prompt = make_system_prompt(
    "You execute Python code and return printed output. "
    "You work with Planner, Research, and Coder colleagues."
)

# --- Create agents ---
planner_agent = create_react_agent(llm, tools=[], prompt=planner_prompt)
research_agent = create_react_agent(llm, tools=[tavily_tool], prompt=research_prompt)
coder_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=coder_prompt)
executor_agent = create_react_agent(llm, tools=[python_repl_tool], prompt=executor_prompt)

# --- Define nodes ---
def planner_node(state: MessagesState) -> Command[Literal["research", END]]:
    result = planner_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "research")
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="planner")
    return Command(update={"messages": result["messages"]}, goto=goto)

def research_node(state: MessagesState) -> Command[Literal["coder", END]]:
    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "coder")
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="researcher")
    return Command(update={"messages": result["messages"]}, goto=goto)

def coder_node(state: MessagesState) -> Command[Literal["executor", END]]:
    result = coder_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "executor")
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="coder")
    return Command(update={"messages": result["messages"]}, goto=goto)

def executor_node(state: MessagesState) -> Command[Literal["planner", END]]:
    result = executor_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "planner")
    result["messages"][-1] = HumanMessage(content=result["messages"][-1].content, name="executor")
    return Command(update={"messages": result["messages"]}, goto=goto)

# --- Setup graph ---
graph = Graph(
    nodes={
        "planner": planner_node,
        "research": research_node,
        "coder": coder_node,
        "executor": executor_node,
    },
    start_node="planner"
)

# --- Run multi-agent loop ---
def run_multi_agent_loop(user_input: str):
    state = MessagesState(messages=[HumanMessage(content=user_input, name="user")])
    current_node = graph.start_node

    while current_node != END:
        node_func = graph.nodes[current_node]
        command = node_func(state)
        state = MessagesState(messages=command.update["messages"])
        current_node = command.goto

    print("Multi-agent process finished.")


if __name__ == "__main__":
    user_question = "Draw line chart for GDP in India for the last decade."
    run_multi_agent_loop(user_question)


PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/model-field-overridden